# Example NP problems published in A. Lucas, *Front. Phys.* (2014) - Graph partitioning problem

This example code implements the **Graph partitioning problem** introduced in the paper [A. Lucas, "Ising formulations of many NP problems", *Front. Phys.* (2014)](https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full) using Fixstars Amplify. Other NP-complete and NP-hard problems introduced in the same paper are also discussed below (the corresponding sections in the paper are shown in the brackets).

- **Graph partitioning problem** (Sec. 2.2).
- [Maximum clique problem](https://amplify.fixstars.com/en/demo/lucas2014_2_3_cliques) (Sec. 2.3)
- [Exact cover problem](https://amplify.fixstars.com/en/demo/lucas2014_4_1_exact_cover) (Sec. 4.1)
- [Set packing problem](https://amplify.fixstars.com/en/demo/lucas2014_4_2_set_packing) (Sec. 4.2)
- [Minimum vertex cover problem](https://amplify.fixstars.com/en/demo/lucas2014_4_3_vertex_cover) (Sec. 4.3)
- [Satisfiability problem (SAT)](https://amplify.fixstars.com/en/demo/lucas2014_4_4_satisfiability) (Sec. 4.4)
- [Minimal maximal matching problem](https://amplify.fixstars.com/en/demo/lucas2014_4_5_minimal_maximal_matching) (Sec. 4.5)
- [Graph coloring problem](https://amplify.fixstars.com/en/demo/lucas2014_6_1_graph_coloring) (Sec. 6.1)
- [Clique cover problem](https://amplify.fixstars.com/en/demo/lucas2014_6_2_clique_cover) (Sec. 6.2)
- [Job sequencing problem with integer lengths](https://amplify.fixstars.com/en/demo/lucas2014_6_3_job_sequencing_with_integer_lengths) (Sec. 6.3)
- [Hamiltonian cycle problem](https://amplify.fixstars.com/en/demo/lucas2014_7_1_hamiltonian_cycles_and_paths) (Sec. 7.1)
- [Directed feedback vertex set problem](https://amplify.fixstars.com/en/demo/lucas2014_8_3_directed_feedback_vertex_set) (Sec. 8.3)
- [Minimum feedback edge set problem](https://amplify.fixstars.com/en/demo/lucas2014_8_5_feedback_edge_set) (Sec. 8.5)
- [Graph isomorphism problem](https://amplify.fixstars.com/en/demo/lucas2014_9_graph_isomorphisms) (Sec. 9)


## Graph partitioning problem

Given a graph $G$ with $2N$ vertices, the **graph partitioning problem** is to find a way to partition $G$ into two sets of $N$ vertices each, such that the number of edges of $G$ connecting $2$ points belonging to different sets is minimized.

For example, in the following graph, if we partition the 8 vertices into a set of 4 orange vertices and a set of 4 blue vertices, there are 2 edges connecting the blue and orange vertices. It is also easy to see that this partition is the optimal solution.


![Graph](../figures/Lucas2014/graph_partition0.png)


This example program uses Fixstars Amplify to solve the graph partitioning problem. The formulation follows that of Sec. 2.2 of [A. Lucas, Front. Phys. (2014)](https://www.frontiersin.org/articles/10.3389/fphy.2014.00005/full).

## Problem definition

First, as an example, create a graph $G$ with $2N$ vertices using NetworkX.

In [ ]:
import networkx as nx
import numpy as np

N = 4  # Half the number of vertices in the graph
G = nx.Graph()
G.add_nodes_from(range(2 * N))
# Define edges connecting two vertices
elist = [(0, 1), (0, 2), (1, 2), (2, 3), (3, 4), (3, 5), (3, 6), (4, 5), (5, 6), (6, 7)]
G.add_edges_from(elist)

pos = nx.circular_layout(G)

nx.draw_networkx(G, node_size=300, font_color="w", pos=pos)

## Formulation

### Decision variables 

Let each of $2N$ binary variables $q$ correspond to each vertex in $G$ to indicate which set each vertex belongs to. For example, if $q=0$ is the vertex set denoted by blue and $q=1$ is the vertex set denoted by orange, the binary variable combinations corresponding to the partitioning in the figure below are shown in the table below.

![図](../figures/Lucas2014/graph_partition1.png)

| Index of vertex | 0   | 1   | 2   | 3   | 4   | 5   | 6   | 7   |
| --------------- | --- | --- | --- | --- | --- | --- | --- | --- |
| $q$             | 0   | 0   | 0   | 1   | 1   | 1   | 1   | 0   |

### Objective function

To solve the graph partitioning problem, we can determine the value of the decision variable $q$ so as to minimize the number of edges connecting vertices belonging to different sets.

For vertices $u$ and $v$ of $G$ to belong to different sets, the exclusive OR (xor) of $q_u$ and $q_v$ must be 1, which is expressed as $-2q_u q_v + q_u + q_v$ in quadratic form. Among all pairs $(u, v)$ of vertices connected by edges, the objective function is the minimum number of $(u, v)$ where $u$ and $v$ belong to different sets. Therefore, the objective function is:

$$
\sum_{(u, v) \in E} \operatorname{xor}(q_u, q_v) = \sum_{(u, v) \in E} -2q_uq_v + q_u + q_v.
$$

### Constraints

For the partition of the vertex set of $G$ represented by the decision variable $q$ to be two sets consisting of $N$ vertices,
it is necessary and sufficient that there are $N$ binary variables, which are $0$ and $1$, respectively. Therefore, the sum of $q_i$ is $N$:

$$
\sum_{i = 0}^{2N-1}q_i = N.
$$


## Implementation

With the problem and formulation described above, let's implement and solve the problem. First, create binary decision variables $q$ using the `BinarySymbolGenerator` in the Fixstars Amplify SDK.

In [ ]:
from amplify import BinarySymbolGenerator

gen = BinarySymbolGenerator()
q = gen.array(2 * N)

Next, we create the objective function $\sum_{(u, v) \in E} \operatorname{xor}(q_u, q_v)$. The logical operator, `q[u] ^ q[v]`, is overloaded for the binary decision variables in Fixstars Amplify SDK, and $\operatorname{ xor}(q_u, q_v)$ is computed to take the same value as a second-order polynomial ($-2q_uq_v + q_u + q_v$).

In [ ]:
cost = sum([q[u] ^ q[v] for u, v in G.edges])

Next, we create a constraint condition. As mentioned above, the constraint condition is that the sum of the $2N$ binary decision variables is exactly $N$.

In [ ]:
from amplify.constraint import equal_to

constraint = equal_to(q, N)

Let us combine the created objective function and constraints into a logical model.


In [ ]:
model = cost + constraint

Now, we define the client and solve the logical model on the Fixstars Amplify Annealing Engine (AE). If the return value of `solve` is not empty, there is a solution that satisfies the constraints.

In [ ]:
from amplify.client import FixstarsClient
from amplify import Solver

client = FixstarsClient()
# client.token = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # If you use Amplify in a local environment or Google Colaboratory, enter your Amplify API token.
client.parameters.timeout = 1000

# Define and execute the solver
solver = Solver(client)
result = solver.solve(model)

if len(result) == 0:
    print("No solution has been found.")
else:
    print("A solution has been found.")

Finally, visualize the resulting graph partition. Since the problem setup is similar to the graph in the introduction, the resulting partition will be similar. You can try changing the shape of the graph and the number of edges to see if you can achieve the correct partition.

In [ ]:
values = q.decode(result[0].values)
colors = [f"C{int(value)}" for value in values]

nx.draw_networkx(G, node_size=600, node_color=colors, font_color="w", pos=pos)